In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
final_results = {}

with open('practice.html', 'rb') as f:
    soup = bs(f, 'html.parser')

    results_info = soup.find_all('p')[0].get_text()
    results_list = results_info.splitlines()

    final_results["date"] = [str(results_list[1].split(': ')[1])]
    final_results["time"] = [str(results_list[2].split(': ')[1])]
    final_results["operator"] = [str(results_list[3].split(': ')[1])]
    final_results["LHDid"] = [str(results_list[4].split(': ')[1])]
    final_results["LHDD"] = [str(results_list[5].split(': ')[1])]
    final_results["L_id"] = [str(results_list[6].split(': ')[1])]
    final_results["L_desc"] = [str(results_list[7].split(': ')[1])]
    final_results["num_channels"] = [int(results_list[8].split(': ')[1])]
    final_results["plate_desc"] = [str(results_list[9].split(': ')[1])]
    final_results["notes"] = [str(results_list[10].split(': ')[1])]

    print(final_results)

{'date': ['03 Jan 2022'], 'time': ['2:10:31 PM GMT-8'], 'operator': ['Name Here'], 'LHDid': ['Generic Equipment ID'], 'LHDD': ['Mfg. Area and Device Name'], 'L_id': ['25uL Artel QC Test'], 'L_desc': ['Plate Layout'], 'num_channels': [384], 'plate_desc': ['384-well MVS Verification Plate'], 'notes': ['Weekly QC']}


In [3]:
    run_stats = soup.find_all("td", {"width":"310", "align":"left"})[0:11]

    temp_list = []
    for i in run_stats:
        if (i.find_next('td').get_text().strip()) is None:
            pass
        else:
           temp_list.append(i.find_next('td').get_text().strip())
    
    final_results["target_vol"] = [int(temp_list[0])]
    final_results["target_sol"] = [str(soup.findAll("td", {"width":"310", "align":"left"})[1].get_text())]
    final_results["data_per_channel"] = [int(temp_list[2])]
    final_results["mean_vol_all_channel"] = [float(temp_list[3].strip('%'))]
    final_results["ri_all_channel"] = [float(temp_list[4].strip('%'))]
    final_results["std_dev_all_channels"] = [float(temp_list[5].strip('%'))]
    final_results["cv_all_channels"] = [float(temp_list[6].strip('%'))]
    final_results["ri_pass_fail_limit"] = [float(temp_list[7].strip('%'))]
    final_results["cv_pass_fail_limit"] = [float(temp_list[8].strip('%'))]
    final_results["channel_final_result"] = [str(temp_list[9])]
    final_results["run_stats_final_result"] = [str(temp_list[10])]

    print(final_results)

{'date': ['03 Jan 2022'], 'time': ['2:10:31 PM GMT-8'], 'operator': ['Name Here'], 'LHDid': ['Generic Equipment ID'], 'LHDD': ['Mfg. Area and Device Name'], 'L_id': ['25uL Artel QC Test'], 'L_desc': ['Plate Layout'], 'num_channels': [384], 'plate_desc': ['384-well MVS Verification Plate'], 'notes': ['Weekly QC'], 'target_vol': [25], 'target_sol': ['Range A'], 'data_per_channel': [4], 'mean_vol_all_channel': [25.754], 'ri_all_channel': [3.02], 'std_dev_all_channels': [0.376], 'cv_all_channels': [1.46], 'ri_pass_fail_limit': [10.0], 'cv_pass_fail_limit': [10.0], 'channel_final_result': ['Failed'], 'run_stats_final_result': ['Passed']}


In [4]:
results_df = pd.DataFrame.from_dict(final_results)
results_df['date'] = pd.to_datetime(results_df['date'])
results_df['time'] = pd.to_datetime(results_df['time']).dt.time
results_df

,date,time,operator,LHDid,LHDD,L_id,L_desc,num_channels,plate_desc,notes,...,target_sol,data_per_channel,mean_vol_all_channel,ri_all_channel,std_dev_all_channels,cv_all_channels,ri_pass_fail_limit,cv_pass_fail_limit,channel_final_result,run_stats_final_result
0,2022-01-03,14:10:31,Name Here,Generic Equipment ID,Mfg. Area and Device Name,25uL Artel QC Test,Plate Layout,384,384-well MVS Verification Plate,Weekly QC,...,Range A,4,25.754,3.02,0.376,1.46,10.0,10.0,Failed,Passed


In [5]:
# These are all your tables
list_of_tables = soup.find_all("table", {"width":"95%"})

# Example for channel_relative_frequency table
crf_df = pd.read_html(str(list_of_tables[5]))[0].drop(labels=0, axis=0).set_index(0)
crf_df = crf_df.applymap(lambda x: float(x.strip('%')))
crf_df
# run whatever you want from here for analysis...

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
0,,,,,,,,,,,,,,,,,,,,,
A,3.00,2.83,2.59,2.80,2.59,2.58,2.88,2.67,2.52,3.41,...,2.99,3.24,3.24,2.71,2.68,2.65,2.81,2.84,2.57,2.58
B,2.84,2.87,2.96,2.94,3.04,2.63,3.05,2.86,3.16,2.90,...,2.87,2.95,3.23,3.25,3.29,3.14,-14.76,3.03,3.15,2.77
C,2.93,2.80,2.97,2.71,3.17,3.17,3.05,-15.66,2.95,3.04,...,2.92,3.12,3.14,3.02,3.02,3.49,3.37,3.01,2.89,3.12
D,2.76,3.27,3.13,3.19,3.19,2.97,3.18,3.05,3.14,3.03,...,3.01,3.27,3.36,3.01,3.07,2.92,2.99,3.17,3.02,2.64
E,2.71,2.95,3.26,3.40,2.98,3.31,3.18,3.16,3.02,3.33,...,2.90,2.82,2.82,3.03,3.27,3.45,2.96,2.69,2.90,3.46
F,2.72,2.95,3.07,3.15,2.95,-0.24,3.35,3.31,3.07,3.17,...,3.25,3.30,3.03,3.30,3.51,3.32,3.08,3.05,3.28,2.65
G,2.84,2.92,3.12,3.38,3.00,3.09,3.25,3.37,3.01,3.56,...,3.08,2.86,3.32,2.85,3.19,3.08,3.12,3.12,2.93,3.48
H,3.09,3.22,3.33,2.98,3.01,2.98,3.23,3.11,3.70,3.21,...,-7.59,2.99,3.49,3.28,3.38,3.07,3.00,3.02,3.03,2.62
I,3.15,3.40,3.34,3.13,3.08,3.23,3.29,3.34,3.38,3.16,...,3.45,3.27,3.07,3.32,3.10,3.07,3.04,2.79,3.00,2.82
